In [161]:
import requests
import pandas as pd
import pymysql
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [162]:
host_name = "luxprac1223.c2hiebiagwyd.ap-northeast-2.rds.amazonaws.com"
username = "admin"
password = "admin123"
database_name = "demo"

In [163]:
db = pymysql.connect(
    host=host_name, 
    port=3306,
    user=username,  
    passwd=password, 
    db=database_name, 
    charset='utf8'
)

In [164]:
cursor = db.cursor()
cursor.execute("set names utf8")
db.commit()

In [190]:
cursor.execute("DROP DATABASE IF EXISTS demo;")
cursor.execute("CREATE DATABASE demo DEFAULT CHARSET=utf8 COLLATE=utf8_bin;")
cursor.execute("USE demo;")
db.commit()


cursor.execute("DROP TABLE IF EXISTS `card_company`;")
cursor.execute("DROP TABLE IF EXISTS `card`;")
db.commit()


cursor.execute("""
CREATE TABLE `card_company` (
    id INT NOT NULL AUTO_INCREMENT,
    name VARCHAR(10) NOT NULL,
    organization VARCHAR(4),
    PRIMARY KEY (id)
) DEFAULT CHARSET=utf8;
""")
db.commit()

# 다섯 번째 SQL 문 실행
cursor.execute("""
CREATE TABLE `card` (
    id INT NOT NULL AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    card_company INT NOT NULL,
    img_url TEXT,
    benefits MEDIUMTEXT,
    benefit_summary MEDIUMTEXT,
    PRIMARY KEY (id),
    FOREIGN KEY (`card_company`) REFERENCES `card_company` (`id`)
) DEFAULT CHARSET=utf8;
""")
db.commit()

In [191]:
bank="KB카드"
code="0301"

In [192]:
cursor.execute("""
INSERT INTO `card_company`(name,organization)
VALUES  ('%s', '%s') 
"""%(bank,code))
db.commit()

In [193]:
new_bank_id = cursor.lastrowid
print(new_bank_id)

1


In [194]:
def cate_page(url,cateIdx):
    html=urlopen(url+cateIdx)
    bs=BeautifulSoup(html,'html.parser')
    links = bs.findAll('div', {'class':'card-box__before'})
    for link in links:
        onclick_value=link.a.get('onclick')
        numbers=onclick_value[21:26]
        print(numbers)
        
        card_html=urlopen('https://card.kbcard.com/CRD/DVIEW/HCAMCXPRICAC0076?mainCC=a&cooperationcode='+numbers)
        card_bs=BeautifulSoup(card_html,'html.parser')
        name=card_bs.find('h1',{'class','tit'}).text
        print(name)

        cursor.execute("SELECT COUNT(*) FROM `card` WHERE name = %s", (name))
        count = cursor.fetchone()[0]

        if count > 0:
            print(f"'{name}' 이미 존재\n\n")
            continue
        

        img=card_bs.find('div',{'class','cardBoxInner'}).img.get('src')
        print(img)

        benefit=""
        benefit_summary=""

        importants=card_bs.find('div',{'class','benefitList1'})
        if importants is not None:
            importants=importants.findAll('li')
            for important in importants:
                important_name=important.find('strong',{'class','tit'}).text
                important_text=important.find('span',{'class','txt'}).text
                if(important_name==" "): benefit_summary+=important_text+"\n"
                else: benefit_summary+="["+important_name+"]"+important_text+"\n"
            benefit_summary=benefit_summary.replace('\'','')
            print(benefit_summary)
            print()

        w=card_bs.find('li',{'id':'topTab1'}).text.replace('\n','').replace(' ','')
        if(w=="상세혜택"): tab='tabCon01'
        else: tab='tabCon02'
        
        summary=card_bs.find('div',{'class':'tabMulti marT20 multiLine2'})
        
        if summary is None:
            content=card_bs.find('div',{'id':tab})
            content_str=str(content)

            split_html = content_str.split('<div class="titArea')

            for split_section in split_html[1:]:
                section_soup = BeautifulSoup('<div class="titArea' + split_section, 'html.parser').text.replace('\n','')
                benefit+=str(section_soup)
                benefit+="\n"
            if(benefit==""):
                hidden_elements = content.find_all(class_='hidden')
                for element in hidden_elements:
                    element.decompose()
                content=content.text.replace('\n','')
                benefit+=content


        else:
            summary=summary.text.replace('\n', '').replace(' ', '')[:5]
            if summary == "서비스요약":
                contents=card_bs.find('div',{'id':tab})
                details=contents.findAll('div', id=re.compile(f'^{re.escape(tab)}'),recursive=False)
                for detail in details[1:]:
                    detail_str=str(detail)
                    split_html = detail_str.split('<div class="titArea')
                    for split_section in split_html[1:]:
                        section_soup = BeautifulSoup('<div class="titArea' + split_section, 'html.parser').text.replace('\n','')
                        benefit+=str(section_soup)
                        benefit+="\n"
                    if(benefit==""):
                        hidden_elements = detail.find_all(class_='hidden')
                        for element in hidden_elements:
                            element.decompose()
                        detail=detail.text.replace('\n','')
                        benefit+=detail

            else:
                contents=card_bs.find('div',{'id':tab})
                details=contents.findAll('div', id=re.compile(f'^{re.escape(tab)}'),recursive=False)
                for detail in details:
                    detail=str(detail)
                    split_html = detail.split('<div class="titArea')
                    for split_section in split_html[1:]:
                        section_soup = BeautifulSoup('<div class="titArea' + split_section, 'html.parser').text.replace('\n','')
                        benefit+=str(section_soup)
                        benefit+="\n"
        benefit=benefit.replace('\'','')
        
        print(benefit)
        print()
        cursor.execute("""INSERT INTO `card`(name,card_company,img_url,benefits,benefit_summary)VALUES  ('%s', %d, '%s', '%s', '%s') """%(name,new_bank_id,img,benefit,benefit_summary))
        db.commit()

In [195]:
url1='https://card.kbcard.com/CRD/DVIEW/HCAMCXPRICAC0047?pageNo=1&cateIdx='
for i in range (1, 12):
    cateIdx = str(i)
    cate_page(url1,cateIdx)
    

09321
알뜰교통플러스 카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09321_img.png
[대중교통 할인]대중교통 버스, 지하철 이용시10% 할인
[생활서비스 할인]생활서비스 5% 할인* 이동통신요금 자동납부* 커피 업종* 편의점 업종* 영화(CGV, 롯데시네마)* 패스트푸드 업종
[KB Pay 이용시 추가할인]‘생활서비스 할인‘ 영역에서 KB Pay 이용시 추가 5% 할인


   청구 할인 서비스    청구 할인 서비스           구분 세부 영역 할인율 월 할인한도 전월 실적     대중교통 할인 버스, 지하철 10% 5천원 30만원   생활서비스할인 이동통신요금 자동납부(SK, KT, LG, LiivM) 건당1만원이상이용 시 5% 5천원   커피 업종   약국 업종   편의점 업종   영화(CGV, 롯데시네마)   패스트푸드 업종   KB Pay 추가할인 생활서비스 할인 영역에서KB Pay 이용 시 추가할인 5% 5천원   합 계 1만5천원      대중교통 : 택시, 시외버스, 고속버스, 공항버스 제외* 버스/지하철 요금할인은 실제 카드 사용일이 아닌 이용대금명세서 상 기재된 이용일을 기준으로 서비스 제공 실적 유예 제공 : 최초 카드 사용 등록 후 다음 달 말일까지 전월 실적 조건 미달 시에도 서비스 제공  


09122
WE:SH All 카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09122_img.png
[국내 가맹점]국내 가맹점1% 할인
[해외 가맹점]해외 가맹점2% 할인
[자동이체 할인]쇼핑 멤버십 50% 할인,OTT 10% 할인,이동통신 5% 할인


  국내 가맹점 1% 할인   전월 이용실적 조건 및 할인한도 없음     
  해외 가맹점 2% 할인   할인한도 : 월 4만원 (전월 이용실적 조건 없음) 해외 할인은 국내외겸용 카드에 한해 제공 해외 이용 시 별도의 수수료 부과되며, 자세

American Express Rose Gold Card
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09116_img.png
[온라인 쇼핑]온라인 쇼핑몰, 오픈마켓KB Pay 이용건 10% 할인
[커피/편의점]커피 오프라인 매장/모바일 앱, 편의점 KB Pay 이용건 10% 할인
[오프라인 쇼핑]대형마트, 백화점, 슈퍼KB Pay 이용건 5% 할인
[온라인 항공/면세점]항공(대한항공/아시아나), 인터넷면세점 KB Pay 이용건 5% 할인


   온라인 쇼핑 10% 할인 (KB Pay 결제 시)    쿠팡, 티몬, 위메프, G마켓, 11번가, SSG.COM                                ※ 상품권, 여행, 항공권, 티켓, 도서, 별도 팝업 사이트 제외      [월 할인한도]  온라인 쇼핑 할인    구분 할인율 전월 이용실적별 월 할인한도   1구간30만원 이상 2구간70만원 이상 3구간120만원 이상     온라인 쇼핑 10% 1만원 1만 5천원 2만원      최초 발급받은 American Express Rose Gold KB Kookmin Card 사용등록일(KB Pay 등 간편결제 등록 포함)로부터 다음달 말일(실적유예기간)까지는 전월 이용실적 30만원 미만시에도 1구간 할인서비스 제공 
   커피/편의점 10% 할인 (KB Pay 결제 시)    커피 오프라인 매장                                ※ 커피전문점 업종 오프라인 결제건에 한하여 할인 ※ 백화점/대형마트 등 일부 입점 매장 제외  스타벅스 사이렌오더, 커피빈 퍼플오더, 메가커피 메가오더                                ※ 공식 모바일 앱 일반 결제건에 한하여 할인  GS25, CU                                ※ 백화점/대형마트 등 일부 입점 매장 제외 ※ 오프라인 매장 일반 결제건에 한함 (온라인 가맹점 제외)      [월

삼성페이 KB국민카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/04434_img.png
[할인서비스]삼성 페이 간편결제10%청구할인
[할인서비스]주요 생활영역5%청구할인


 삼성 페이 할인서비스 서비스 요약 테이블구분할인율월 할인 한도확인사항삼성 페이 할인10%15,000원삼성 페이로 건당 3만원 이상 결제 시 청구 할인삼성 PAY카드 할인서비스는 전월 이용실적 40만원 이상 시 혜택이 제공됩니다.최초 카드 사용등록일로부터 다음 달 말일까지는 전월 이용실적이 없는 경우에도 할인서비스가 제공됩니다.삼성 페이 이용약관 등 세부사항은 삼성 페이 App에서 확인할 수 있습니다.
 주요 생활 할인서비스 서비스 요약 테이블구분할인율월 할인 한도확인사항주요 생활 할인5%5,000원- 할인대상 가맹점에서 건당 1만원 이상 결제 시 청구 할인												- 할인대상 가맹점 : 이동통신(SKT, KT, LGU+)*, 커피전문점 업종, 												약국업종, 편의점업종, 영화(CGV, 롯데시네마), 패스트푸드업종												* 이동통신요금 자동납부 시 할인삼성 PAY카드 할인서비스는 전월 이용실적 40만원 이상 시 혜택이 제공됩니다.최초 카드 사용등록일로부터 다음 달 말일까지는 전월 이용실적이 없는 경우에도 할인서비스가 제공됩니다.삼성 페이 이용약관 등 세부사항은 삼성 페이 App에서 확인할 수 있습니다.주요 생활 할인 할인대상 가맹점은 해당 가맹점 사업자번호로 당사 영업점 및 고객센터(☎1588-1688), KB국민은행 영업점을 통해 확인이 가능합니다.


09157
가온올림카드(실속형)
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09157_img.png
[기본적립]국내 가맹점 0.7% 기본적립
[주말/공휴일]국내 가맹점 0.5% 추가적립
[음식/커피]음식점, 커피 전문점 업종 0.5% 추가적립
[교통]버스/지하철, 

금융포인트리카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09040_img.png
[금융포인트]0.5~1% 기본적립
[금융포인트]3~4% 특별적립
[금융]KB국민은행 대출금리연 0.1~0.3%p 할인
[금융]다양한 KB금융서비스 금융수수료 면제 서비스


   금융포인트 금융포인트 0.5~5% 적립      KB국민 금융포인트리카드 결제납부실적에 따라 금융포인트 0.5~5% 적립 	 KB국민 금융포인트리카드 결제납부실적에 따라 금융포인트 0.5~5% 적립    당월 결제납부실적 기본 적립 특별 적립     100만원 미만 기본 0.5% 3%   100만원 이상 기본 0.5% + 추가 0.5% (추가는 100만원 초과금액에 한함) 4%     당월 결제납부실적 기준 : 당월 1일~말일까지 회수된 KB국민 금융포인트리카드의 일시불 및 할부 결제원금 기준 기본 적립 제외대상 : 지방세, 대학(대학원) 등록금, 선불카드 구입 · 충전금액 특별 적립대상은 3대 업종 이용 시 제공되며 월 20만원 이용금액까지 적립됩니다. 	 3대 업종 : 주유소 업종(충전소 제외), 통신(전화요금 업종 및 인터넷 이용요금 업종), 3대 할인점(이마트, 롯데마트, 홈플러스)         
   금융포인트 KB증권 매매수수료의 5% 금융포인트 적립      KB plustar 통장 연계 KB증권 증권 거래 시 매매수수료의 5% 금융포인트 적립 	 금융포인트 적립 기준 : 유효한 KB국민 금융포인트리카드 및 KB plustar 통장을 소지한 고객으로 KB plustar 통장과 연계된 KB증권의 증권계좌를 통하여 증권거래(매수 및 매도) 시 매매수수료의 5% 금융포인트 적립(KB증권 주식거래 수수료율 등 세부내역은 KB증권 홈페이지에서 확인 바랍니다.) 금융포인트 적립 시기 : 월 1회, 증권거래(매수 및 매도) 시점 기준으로 익월 영업일이 있는 첫째 주 토요일에 일괄 적립 KB국민 금융포인트리카드 소지 회원에 한하여 제공(

LFmall 신용카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09314_img.png
[할인서비스]LF mall 할인최대2만원 청구할인
[적립서비스]국내외 가맹점 적립 서비스0.5% 포인트리 적립


   LFmall (lfmall.co.kr) 이용 시 5% 청구할인    LFmall 청구할인 안내표         구분 할인율 전월 이용실적&월 할인한도   30만원 미만 30만원 이상     LFmall 청구할인 5% 5천원 2만원    할인 제외 : 무이자할부, 상품권 및 선불카드(선불전자지급수단 포함) 구입/충전금액, LF오프라인 매장 이용금액 실적 유예 : 최초 카드 사용등록일부터 다음달 말일(실적 유예기간)까지는 ‘LFmall 신용카드’ 전월이용실적이 30만원 미만인 경우에도 30만원 이상에 해당하는 서비스가 제공됩니다. 
   포인트리 적립 서비스    국내외 가맹점 이용 시 0.5% 포인트리 적립 (전월 이용실적 조건 및 적립한도 제한 없음)            LFmall 이용금액은 적립 제외 (세부사항은 ‘적립서비스 제외 대상‘ 참조)   [포인트리 적립 확인 사항]  KB국민카드 포인트리 1점은 1원과 가치가 동일합니다. 포인트리적립은 해당 이용분의 신용공여기간에 따른 신용카드 결제일 다음 영업일에 적립됩니다. 적립 시점 기준 60개월이 경과한 포인트리는 적립 순서에 따라 월 단위로 소멸됩니다. 포인트리 활용에 대한 자세한 내용은 KB국민카드 홈페이지(www.kbcard.com)>혜택>포인트리에서 확인하실 수 있습니다. 


09115
American Express Blue Card
'American Express Blue Card' 이미 존재


09116
American Express Rose Gold Card
'American Express Rose Gold Card' 이미 존재


04361
이마트II KB국민카드(옐로우)
https://img1.kbcard.com/S

...

알뜰폰Hub 카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/04323_img.png
[할인서비스]알뜰폰 통신료 자동이체1만원~1만5천원  청구할인
[티타늄 서비스]공항라운지공항/호텔 발레파킹


통신알뜰폰 통신료 자동이체 할인 서비스  알뜰폰Hub 카드 서비스 요약         구 분 세부 내용 전월 이용실적에 따른 할인 금액   1구간(50만원 이상) 2구간(100만원 이상)     자동이체 할인 알뜰폰 통신료 자동이체 할인 1만원 1만5천원   티타늄 서비스 공항/호텔 발레파킹, 공항 라운지 등 전월 이용실적 30만원 이상 시    최초 카드 사용 등록일로부터 다음달 말일까지는 알뜰폰Hub 카드 실적이 없는 경우에도 1구간(전월 이용실적 50만원 이상)에 해당하는 10,000원 청구할인 서비스가 적용됩니다. 할인금액보다 자동이체 승인금액이 작을 경우 승인금액만큼 할인 적용됩니다. 통신료 자동이체 승인 건이 여러 건인 경우 합산하여 월 할인금액 내에서 할인 됩니다. 알뜰폰 사업자가 기존 통신사(SKT, KT, LG U+)를 통하여 통신 요금 청구 시 기존 통신사(SKT, KT, LG U+)의 청구 요금과 통합 할인이 적용됩니다. 알뜰폰Hub 카드를 K-World(UPI) 브랜드로 발급 시 티타늄 등급 브랜드 서비스가 제공됩니다. 해당 할인서비스는 알뜰폰Hub 카드로 통신료 자동이체 등록 시에만 제공되며, 카드 수령 후 개별 알뜰폰 고객 콜센터에서 자동이체를 별도로 신청하시기 바랍니다. (자동이체 등록된 통신료가 카드사에 청구되는 시점부터 청구 할인 서비스 적용) 


04249
T-Premium KB국민카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/04249_img.png
[할인서비스]SKT통신요금 자동납부기본할인5천원~1만원 청구할인
[할인서비스]SKT통신요금 자동납부 추가할인1만원~1만1천원 청구할인


SKT통신요금 

04418
SK매직 올림카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/04418_img.png
[할인 서비스]SK매직 렌탈료1만1천원 ~ 2만원 청구할인
[할인 서비스]메가박스/CGV   3,500원 청구할인
[할인 서비스]놀이공원30~50% 현장할인


   							SK매직 렌탈료 자동이체 시 1만 1천원~2만원 청구할인						     						해당 할인서비스는 SK매직 렌탈료 자동이체 신청시에만 제공되며, 카드 수령 후 SK매직 콜센터 (☎ 1577-7784)에서 자동이체를 별도로 신청하시기 바랍니다.					 KB국민 SK매직 올림카드 전월 이용실적별 할인 서비스 제공                       전월 이용실적 30만원이상 이용 시 1만 1천원 청구할인 전월 이용실적 70만원이상 이용 시 1만 5천원 청구할인 전월 이용실적 100만원이상 이용 시 2만원 청구할인  할인한도보다 자동이체 승인된 금액이 작을 경우 승인금액만큼 할인 적용 SK매직 렌탈료 자동이체 승인 건이 여러 건인 경우 월 할인한도 내에서 합산하여 할인 최초 카드 사용등록일로부터 다음달 말일까지 「KB국민 SK매직 올림카드」 이용실적이 없는 경우에도 렌탈료 1만 1천원 할인 제공 
   메가박스/CGV 온라인 영화 예매 시 3,500원 할인적용     메가박스/CGV 온라인 영화 예매 시 승인 건당 3,500 원 청구 할인 1일 2회, 월 4회, 연 10회 이내 KB국민 SK매직 올림카드 전월 이용실적 30만원 이상 시 할인서비스 제공 최초 카드 사용등록일로부터 다음달 말일까지 「 KB국민 SK매직 올림카드 」 이용실적이 없는 경우에도 할인 제공 
   롯데월드 50% 현장 할인     						롯데월드 어드벤처 종합이용권 50% 현장 할인						 회원별 일 1매, 월 2매, 연간 5매 할인제공(가족카드 할인 미제공)   KB국민 SK매직 올림카드 전월 이용실적 30만원 이상 시 할인서비스 제공 최

웰릭스렌탈Ⅱ카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/04326_img.png
[할인 서비스]웰릭스렌탈 자동납부요금 1만2천원/1만7천원 청구할인


 웰릭스렌탈 자동납부요금 청구할인 KB국민 웰릭스렌탈Ⅱ 카드 서비스 요약구 분전월 이용실적에 따른 월간 할인 금액1구간(30만원 이상)2구간(70만원 이상)웰릭스렌탈 요금 자동납부1만2천원1만7천원최초 카드 사용 등록일로부터 다음달 말일까지는 KB국민 웰릭스렌탈Ⅱ 카드 전월 이용실적이 없는 경우에도 1구간(전월 이용실적 30만원 이상)에 해당하는 할인 서비스가 적용됩니다.할인금액보다 자동납부 승인금액이 작을 경우 승인금액만큼 할인 적용됩니다.(예시) 전월 이용실적이 30만원 이상, 자동납부 승인금액이 9,000원인 경우 9,000원이 할인됩니다.웰릭스렌탈 자동납부 승인 건이 여러 건인 경우 합산하여 월 할인한도 내에서 할인 됩니다.KB국민 웰릭스렌탈Ⅱ카드로 웰릭스렌탈 요금을 자동납부하시는 경우에 한하여 할인이 제공되며, 카드 수령 후 웰릭스렌탈 고객센터에서 자동납부를 별도로 신청하시기 바랍니다.


04325
쿠쿠렌탈Ⅱ카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/04325_img.png
[할인 서비스]쿠쿠렌탈 자동납부요금1만2천원/1만7천원  청구할인


 쿠쿠렌탈 자동납부요금 청구할인 KB국민 쿠쿠렌탈 카드 서비스 요약구 분전월 이용실적에 따른 월간 할인 금액1구간(30만원 이상)2구간(70만원 이상)쿠쿠렌탈 요금 자동납부1만2천원1만7천원최초 카드 사용 등록일로부터 다음달 말일까지는 KB국민 쿠쿠렌탈Ⅱ 카드 전월 이용실적이 없는 경우에도 1구간(전월 이용실적 30만원 이상)에 해당하는 할인 서비스가 적용됩니다.할인금액보다 자동납부 승인금액이 작을 경우 승인금액만큼 할인 적용됩니다.(예시) 전월 이용실적이 30만원 이상, 자동납부 승인금액이 9,000원인 경우 9,000원

국방멤버쉽카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/00996_img.png
[주유]GS칼텍스 리터당 70원 청구할인
[레저]군체력단련장/휴양시설 10% 할인
[포인트]모든 가맹점 0.2% 포인트리 적립 국군복지단 매장에서 이용시 추가포인트리 0.3% 적립
[무이자할부]국군복지단 매장/백화점/할인점 2~3개월 무이자할부


   KB국민카드 회원자격 대상자로서 다음에 해당하는 자  현역군인(장교 및 부사관) 및 군무원, 근무원, 국방부 및 산하기관 일반직 공무원 등     10년 이상 장기근속 예비역 사관/부사관 후보생, 국가유공자, 중사 이상 복무 후 전역한 예비역 10년 이상 복무 후 퇴직한 군무원   
   신청방법 국군복지단 홈페이지에서 카드발급신청 가능      국군복지단 홈페이지에서 카드발급신청 가능 징구 서류 : 국방멤버쉽카드의 성격상 카드발급 대상이 제한되므로 다음과 같이 처리됩니다.							 하사 계급 이상 현역 군인, 군(근)무원 등 현 재직자 : 유선 재직확인후 발급이 가능하며, 확인불가시 신분증 등 (군인신분증, 예비역신분증)을 확인 후 발급 10년 이상 장기근무 예비역 : 장기근무 확인 서류을 확인한 후 발급   
   국군복지단 할인서비스 군 휴양시설 및 체력단련장 이용시 10% 청구할인 (월 이용금액 20만원까지)      군 휴양시설은 객실요금, 체력단련장은 골프 이용료에 대한 할인 적용함							동 서비스 내용은 당사 및 제휴사의 사정에 따라 사전 안내 후 변경될 수 있습니다.     
   국군복지단 매장 무이자할부 서비스 2~3개월 무이자할부     국군복지단 매장 상시 2~3개월 무이자할부 서비스 제공   
   주유할인 서비스  GS칼텍스 주유소에서 주유시 L당70원 할인      GS칼텍스 주유소에서 주유시 L당70원 할인 L당 할인금액(70원)은 GS칼텍스 본사에서 고시하는 휘발유가 기준임 대상유류 : 휘발유, 경유, 등유에 한함(LPG제외)

보탬e전용카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09084_img.png
   KB국민 보탬e전용카드란?  지방보조금 통합관리시스템(보탬e)에서 보조금 처리(조회, 정산)가 가능한 보조금 전용 카드 KB국민 보탬e전용카드의 발급내역 및 사용내역을 지방보조금관리시스템(www.losims.go.kr)에 전송하여 지방보조금의 투명하고 편리한 집행을 지원 발급대상  지방보조금 보조사업자(지자체로부터 보조금을 지원 받는 개인) 복수사업 수행시, 별도브랜드로 추가발급 필요  보조사업별 1개 카드가 매칭 가능하므로, 복수 사업수행으로 복수 카드 필요시, 브랜드(로컬, 마스터, 비자, JCB, K-WORLD(UPI)) 별 1매씩 추가 발급하여 주십시오. 상품 서비스  카드사용알림서비스(SMS) 수수료 면제 카드 신청기관/업체 앞 적립 포인트 없음 교통기능 미탑재 이용 안내  일시불 거래만 가능 : 할부, 단기카드대출(현금서비스), 장기카드대출(카드론), 리볼빙 등 거래 불가 클린카드 기능 : 보탬e에서 지정하는 제한업종 사용불가   이용안내       구분 제한업종     유흥업종 ‘한국표준산업분류’에 따라 접객요원을 두고 술을 판매하는 일반유흥주점, 무도시설을 갖추고 술을 판매하는 무도유흥주점   위생업종 이·미용실, 피부미용실, 사우나, 안마시술소, 발마사지, 스포츠마사지, 네일아트, 지압원 등 대인서비스   레저업종 골프장, 골프연습장, 스크린골프장, 노래방, 사교춤, 전화방, 비디오방, 당구장, 헬스클럽, PC방, 스키장   사행업종 카지노, 복권방, 오락실   기타업종 성인용품점, 총포류 판매점  

04320
밀크T 카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/220608_04320_img.png
[할인서비스]밀크T 학습비 자동이체 1만 2천원/1만 7천원 청구할인


  밀크T 학습비 자동이체 시 전월 실적에 따라 청구

In [196]:
url2='https://card.kbcard.com/CRD/DVIEW/HCAMCXPRICAC0056?pageNo=1&cateIdx='
for i in range (1, 9):
    cateIdx = str(i)
    cate_page(url2,cateIdx)

09322
알뜰교통플러스 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09322_img.png
[대중교통 적립]대중교통 버스, 지하철 이용시10% 적립(월 적립한도 2천점)
[생활서비스 적립]생활서비스 1% 적립(월 적립한도 4천점)* 이동통신요금 자동납부* 커피업종* 편의점 업종* 영화(CGV, 롯데시네마)* 패스트푸드 업종* 건당 1만원 이상 이용 시 적립
[KB Pay 이용시 추가적립]‘생활서비스 할인‘ 영역에서 KB Pay 이용시 추가 1% 적립(월 적립한도 4천점)


   포인트리 적립 서비스    포인트리 적립 서비스           구분 세부 영역 적립율 월 적립한도 전월 실적     대중교통 적립 버스, 지하철 10% 2천점 20만원   생활서비스적립 이동통신요금 자동납부(SK, KT, LG, LiivM) 건당1만원이상이용 시 1% 4천점   커피 업종   약국 업종   편의점 업종   영화(CGV, 롯데시네마)   패스트푸드 업종   KB Pay 추가적립 생활서비스 적립 영역에서KB Pay 이용 시 추가적립 1% 4천점   합 계 1만점      대중교통 : 택시, 시외버스, 고속버스, 공항버스 제외* 버스/지하철 요금적립은 실제 카드 사용일이 아닌 이용대금명세서 상 기재된 이용일을 기준으로 서비스 제공 실적 유예 제공 : 최초 카드 사용 등록 후 다음 달 말일까지 전월 실적 조건 미달 시에도 서비스 제공  


09620
위메프페이 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09620_img.png
[위메프포인트]위메프페이결제  1% 적립
[위메프포인트]위메프오1% 적립
[위메프포인트]GS25편의점 2% 적립
[위메프포인트]커피제과업종5% 적립
[위메프포인트]구독서비스업종10% 적립


   위메프포인트 위메프페이결제 1%적립       위메프페이 체크카드         구분 적립률 월간 적립

H.Point 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/04906_img.png
[H.Point]국내  가맹점 0.2% 기본적립
[H.Point]현대백화점 그룹 0.5% 추가적립 
[제과]파리바게뜨 5% 환급할인
[커피]스타벅스 5% 환급할인
[영화]CGV 5% 환급할인
[뷰티]올리브영 5% 환급할인


제과파리바게뜨  5% 환급할인파리바게뜨서비스 구분할인율전월 이용실적에 따른월간 할인한도1구간(30만원 이상50만원 미만)2구간(50만원 이상)파리바게뜨5%1천원3천원제공조건 : 이용금액 건당 5천원 이상 이용 시 할인실적유예기간 미적용 (전월 이용실적 충족 시에만 제공)  
커피스타벅스  5% 환급할인스타벅스서비스 구분할인율전월 이용실적에 따른월간 할인한도1구간(30만원 이상50만원 미만)2구간(50만원 이상)스타벅스5%1천원3천원실적유예기간 미적용 (전월 이용실적 충족 시에만 제공)  
영화CGV  5% 환급할인CGV서비스 구분할인율전월 이용실적에 따른월간 할인한도1구간(30만원 이상50만원 미만)2구간(50만원 이상)CGV5%1천원3천원제공조건 : 이용금액 건당 1만원 이상 시 할인CGV 영화관, 홈페이지 및 공식 App 예매 시 할인할인제외 : 상품권 구매 및 매점 이용금액실적유예기간 미적용 (전월 이용실적 충족 시에만 제공)  
뷰티올리브영  5% 환급할인올리브영서비스 구분할인율전월 이용실적에 따른월간 할인한도1구간(30만원 이상50만원 미만)2구간(50만원 이상)올리브영5%1천원3천원제공조건 : 이용금액 건당 1만원 이상 시 할인실적유예기간 미적용 (전월 이용실적 충족 시에만 제공)  


01928
정 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/01928_img.png
[백화점]롯데·현대·신세계 7% 환급할인
[홈쇼핑/온라인쇼핑]GS홈쇼핑·CJ홈쇼핑G마켓·옥션 5% 환급할인
[해외이용]해외이용(해외직

그린재킷 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09613_img.png
[[Golf]]골프연습장/스크린골프5% 환급할인
[[Golf]]골프관광(그린재킷투어)5% 환급할인
[[Golf]]골프웨어/골프용품 5% 환급할인
[[Life]]주유소 5% 환급할인
[[Life]]보험료 자동납부 5% 환급할인
[[Life]]해피포인트 멤버쉽  (현장적립)


[Golf]골프연습장/스크린골프 5% 환급할인  KB국민그린재킷 체크카드 골프연습장/스크린골프         구분 전월 이용실적에 따른할인율 및 할인금액   1구간(30만원 이상 60만원 미만) 2구간(60만원 이상)     그린재킷 골프연습 할인 할인율 5% 5%   월 최대 할인가능 이용금액 10만원 10만원   실적조건 : KB국민 그린재킷 체크카드로 전월 30만원 이상 이용 시 제공 실적유예기간 적용 : 최초 카드 사용등록일로부터 다음달 말일까지 서비스 제공 월 최대 10만원 이용금액까지 할인 제공 할인업종 : 골프연습장 (KB국민카드 업종 분류 기준) 이용대금명세서상 기재 된 이용일 기준 제공
[Golf]골프관광(그린재킷 투어) 5% 환급할인  KB국민그린재킷 체크카드 골프관광         구분 전월 이용실적에 따른할인율 및 할인금액   1구간(30만원 이상 60만원 미만) 2구간(60만원 이상)     그린재킷 골프관광 할인 할인율 5% 5%   월 최대 할인가능 이용금액 10만원 10만원   실적조건 : KB국민 그린재킷 체크카드로 전월 30만원 이상 이용 시 제공 실적유예기간 미적용 월 최대 10만원 이용금액까지 할인 제공 직영 홈페이지를 통해 예약 결제 시 할인 제공 할인가맹점 : 그린재킷 투어 (맞춤형 골프전문 여행사) 이용대금명세서상 기재 된 이용일 기준 제공
[Golf]골프웨어/골프용품 5% 환급할인  KB국민그린재킷 체크카드 골프웨어         구분 전월 이용실적에 따른할인율 및 할인금액   1구간(30만원 이상

노리2 체크카드(Global)
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/07972_img.png
[[일상] 커피 할인]스타벅스, 커피빈 10% 할인(월 할인한도 : 3천원)
[[일상] 모바일 할인]구글플레이스토어, 앱스토어 10% 할인(월 할인한도 : 5천원)
[[일상] 문화 할인]인터파크 티켓 10% 할인(월 할인한도 : 7천원)
[[일상] 뷰티 할인]올리브영, 미용실 업종 5% 할인(월 할인한도 : 2천원)
[[일상] 편의점 할인]GS25, CU 5% 할인 5% 할인(월 할인한도 : 2천원)
[[일상] 구독 할인]넷플릭스, 유튜브프리미엄 1만원 이상 시 1,000원 할인(월 할인한도 : 2천원)
[[일상] 배달 할인]배달의민족, 요기요1만원 이상 시 1,000원 할인(월 할인한도 : 1천원)
[[일상] 이동통신 할인]SKT, KT, LG U , Liiv M5만원 이상 시 2,500원 할인(월 할인한도 : 2천5백원)
[[일상] 영화 할인]CGV 1만원 이상 시4,000원 할인(월 할인한도 : 8천원)
[[일상] 놀이공원 할인]에버랜드, 롯데월드 3만원 이상 시 15,000원 할인(월 할인한도 : 1만5천원)
[[Global] 해외이용 할인]해외 가맹점 결제 시 2% 할인(월 할인한도 : 5천원)
[[Global] 공항라운지]전 세계 공항라운지 더라운지멤버스 이용권(연간 1회, 본인만)


  [일상] 커피 할인 스타벅스, 커피빈 10% 할인     커피 할인        구분 할인율 확인사항     스타벅스, 커피빈 10%   전월 이용실적 조건 없음 건당 이용조건 없음 월 할인한도 : 3천원 (월 3만원 이용금액까지 할인) 스타벅스는 사이렌오더 결제 포함 할인제외 : 상품권/선불카드(선불전자지급수단 포함) 구입 및 충전, 백화점, 대형마트, 철도·역사, 호텔 등에 입점한 가맹점 등       본인회원 기준으로 월간 통합할인한도 내에서 혜택이 제공됩니다. 자세한 사항은 상품설명서(약관)를 

우리동네 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/09644_img.png
[우리동네 세탁소]세탁소 업종 1만원 이상 결제 시 10% 환급할인(할인한도: 2천원)
[우리동네 빵집]제과아이스크림 업종 1만원 이상 결제 시 5% 환급할인(할인한도: 2천원)
[우리동네 반찬가게]기타식품 업종 1만원 이상 결제 시 5% 환급할인(할인한도: 2천원)
[우리동네 슈퍼마켓]슈퍼마켓 업종 2만원 이상 결제 시 5% 환급할인(할인한도: 2천원)
[우리동네 야채과일가게]농수축협직판장, 농수축산물점 업종 2만원 이상 결제 시 5% 환급할인(할인한도: 2천원)
[우리동네 취향거래]번개장터 앱(app)에서 3만원 이상 결제 시 5% 환급할인(할인한도: 5천원)
[우리동네 정육점]정육점 업종 3만원 이상 결제 시 10% 환급할인(할인한도: 5천원)


  [우리동네 세탁소] 세탁소 업종 결제 시 10% 환급할인     세탁소 환급할인        구분 할인율 제공조건     세탁소 업종 결제 시 10% 1만원 이상 결제 시   월 할인한도 2천원 -     실적조건 : KB국민 우리동네 체크카드로 전월 20만원 이상 이용 시 제공 월 할인한도 : 2천원 KB국민 우리동네 체크카드로 세탁소 업종에서 1만원 이상 결제 시 할인 제공 KB국민카드 업종 분류 기준, 지정 업종에 한하여 제공             * KB국민카드 홈페이지(PC, 모바일) 카드이용 관련 메뉴에서 매출전표별 가맹점 업종코드 확인 가능         타사 간편결제 이용 시 할인 제외 실적유예기간 미적용 
  [우리동네 빵집] 제과아이스크림 업종 결제 시 5% 환급할인     빵집 환급할인        구분 할인율 제공조건     제과아이스크림 업종 결제 시 5% 1만원 이상 결제 시   월 할인한도 2천원 -     실적조건 : KB국민 우리동네 체크카드로 전월 20만원 이상 이용 시 제공 월 할인한도 : 2천원 KB국민 우리동네 체크카드로

첵첵 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/01914_img.png
[대중교통]버스/지하철 2~4천원 청구할인
[편의점]CU편의점 2~4천원 환급할인
[커피]스타벅스(사이렌오더포함) 2~4천원 환급할인
[영화]CGV 2~4천원 환급할인
[쇼핑]텐바이텐 2~4천원 환급할인
[온라인간편결제]네이버/카카오/SSG페이 2~4천원 환급할인
[뷰티]올리브영/안경(렌즈)점 2~4천원 환급할인
[온라인서점]YES24/교보문고 2~4천원 환급할인
[문화]인터파크티켓 2~4천원 환급할인


대중교통버스·지하철 2~4천원 청구할인대중교통서비스 구분건당 할인금액 및 영역별 할인한도1구간(30만원 이상60만원 미만)2구간(60만원 이상)첵첵 대중교통 할인건당 할인금액1천원2천원영역별 할인한도2천원4천원이용금액 3만원 이상 시 할인 적용실적조건: KB국민 첵첵 체크카드로 전월 30만원 이상 이용 시 제공 (동일계열카드 첵첵, 위글위글 첵첵 실적 합산)실적유예기간 적용: 최초 카드 사용등록일로부터 다음달 말일까지 1구간 내에서 서비스 제공이용대금명세서상 기재 된 이용일 기준 제공
편의점CU편의점 2~4천원 환급할인편의점서비스 구분전월 이용실적에 따른 건당 할인금액 및 영역별 할인한도1구간(30만원 이상60만원 미만)2구간(60만원 이상)첵첵 편의점 할인건당 할인금액1천원2천원영역별 할인한도2천원4천원이용금액 1만원 이상 시 할인 적용실적조건: KB국민 첵첵 체크카드로 전월 30만원 이상 이용 시 제공 (동일계열카드 첵첵, 위글위글 첵첵 실적 합산)실적유예기간 적용: 최초 카드 사용등록일로부터 다음달 말일까지 1구간 내에서 서비스 제공상품권, 선불카드 구입(충전) 금액은 할인 제외
커피스타벅스(사이렌오더 포함) 2~4천원 환급할인커피서비스 구분전월 이용실적에 따른 건당 할인금액 및 영역별 할인한도1구간(30만원 이상60만원 미만)2구간(60만원 이상)첵첵 커피 할인건당 할인금액1천원2천원영역별 할인한도2천원4

노리 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/01664_img.png
[대중교통]버스/지하철 10% 청구할인
[이동통신]이동통신요금 2,500원 환급할인
[영화]CGV 35% 환급할인
[외식]아웃백·VIPS 20% 환급할인
[커피]스타벅스 20% 환급할인
[놀이공원]에버랜드/롯데월드50% 환급할인
[편의점]GS25 5% 환급할인
[서점]교보문고 5% 환급할인
[금융]금융수수료 면제 서비스


대중교통버스·지하철 10% 청구할인월 이용금액 2만원까지 할인대중교통 할인 대상 : RF교통기관 업종(KB국민카드 가맹점 업종 기준)실적 조건 : 전월 이용실적 30만원 이상실제 카드이용일이 아닌 이메일 이용내역서상 기재된 이용일 기준으로 할인 적용 됩니다.전월이용실적에 따른 통합할인한도 적용전월이용실적에 따른 통합할인한도 테이블전월이용실적20만원이상30만원이상 50만원이상100만원이상월간 통합할인한도1만원2만원3만원5만원
통신이동통신요금 5% 환급할인이동통신요금 자동납부 시 5% 환급 할인건당 5만원 이상 이용 시 2,500원 정액할인해당카드로 자동이체 등록 시 할인 적용전월 이용실적 30만원 이상 시 할인 적용유무선 통신요금 통합청구금액 자동이체는 할인 제외SKT, KT Olleh, LGU+에 한하여 할인 적용전월이용실적에 따른 통합할인한도 테이블전월이용실적20만원이상30만원이상50만원이상100만원이상월간 통합할인한도1만원2만원3만원5만원
영화CGV 35% 환급할인이용금액 건당 1만원~2만원까지 할인 적용(1회 최대 할인금액 7,000원)제공조건 : 현장구매 및 CGV 홈페이지(www.cgv.co.kr)·CGV 앱 결제할인제외 : 상품권 구매 / 매점 이용금액최초 발급 후 사용등록일로부터 60일 까지는 월간 통합할인한도 1만원 이내에서 할인 제공됩니다.KB국민 노리체크카드 동일 계열 카드 중복 소지 시 동일계열 카드 중 최초 발급된 카드의 사용등록일을 기준으로 합니다. 전월이용실적에 따른 통

샘 쏘영 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/01574_img.png
[스트리밍]멜론, YouTube Premium 5% 환급할인
[문구점]문구점(팬시용품점) 5%  환급할인
[독서실]독서실(스터디카페) 5%  환급할인
[패스트푸드]패스트푸드 5% 환급할인
[편의점]GS25, CU편의점 5% 환급할인
[대중교통]버스, 지하철 5% 청구할인


스트리밍멜론, YouTube Premium 5% 환급할인멜론, YouTube Premium 5%  환급할인 테이블구분대상가맹점할인율할인한도할인조건(건당최소이용금액)#숨쉬듯 스트리밍멜론YouTube Premium5%1천원3천원 이상 이용 시실적조건: KB국민 샘 쏘영 체크카드로 전월 5만원 이상 이용 시 제공실적유예기간 미적용YouTube Premium, YouTube Music Premium 할인 적용멜론 음원 구독 외 공연 이용 금액 등은 할인 제외상품권, 선불카드 구입(충전) 금액은 할인 제외
문구점업종(팬시용품점)문구점(팬시용품점) 5% 환급할인문구점/독서실  환급할인 테이블구분대상가맹점할인율할인한도할인조건(건당최소이용금액)#내맘대로다꾸문방구업종(팬시용품점 포함)5%1천원3천원 이상 이용 시#오늘도빡공독서실업종(스터디카페 포함)5천원 이상 이용 시KB국민카드 업종 분류 기준 (작성기준일: 2020년 7월 22일)문구점 업종 등록 대표 가맹점: 교보핫트랙스, 아트박스, 알파문구, 모닝글로리, 드림디포 등독서실 업종 등록 대표 가맹점: 토즈스터디센터, 어썸팩토리 등상기 대표 가맹점 중 업종코드 상이한 가맹점은 할인 제외 (예를 들어 ‘커피전문점’ 업종으로 등록 된 토즈 독서실은 업종 상이로 할인 불가)실적조건: KB국민 샘 쏘영 체크카드로 전월 5만원 이상 이용 시 제공실적유예기간 적용: 최초 카드 사용등록일로부터 다음달 말일까지 할인한도 내에서 서비스 제공문구점과 독서실 할인은 KB국민카드 업종 분류 기준, 지정 업종에 한하여 제공됩니다

음 체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/01932_img.png
[외식]한식 업종 5% 평일 환급할인
[커피]커피전문점 업종 5% 평일 환급할인
[제과/아이스크림]제과점, 아이스크림전문점 5% 평일 환급할인
[대중교통]버스,지하철 5% 청구할인


외식한식 업종 5% 평일 환급할인한식업종 환급할인구분할인율전월 이용실적에 따른 월 할인한도1구간(30만원 이상)2구간(50만원 이상)한식5%5천원1만원이용금액 건당 2만원~5만원까지 할인 적용1일 1회, 최대 2,500원까지 월 할인한도 내에서 할인 적용 (주말, 공휴일 제외)한식, 휴게음식점 업종할인제외 : 일식, 중식, 양식, 뷔페, 패밀리레스토랑, 주점 등최초 카드발급 후 사용등록일로부터 다음 달 말일까지 이용실적이 없는 경우에도 1구간 서비스 제공됩니다.
								최초 카드발급 후 사용등록일로부터 다음 달 말일까지 이용실적이 없는 경우에도 1구간 서비스 제공됩니다. 이상)2구간(50만원 이상)커피5%5천원1만원이용금액 건당 1만원~5만원까지 할인 적용1일 1회, 최대 2,500원까지 월 할인한도 내에서 할인 적용 (주말, 공휴일 제외)커피전문점 업종 (스타벅스, 커피빈, 엔제리너스, 파스쿠찌 등)
제과·아이스크림제과점·아이스크림 전문점 5% 평일 환급할인제과점·아이스크림전문점 환급할인구분할인율전월 이용실적에 따른 월 할인한도1구간(30만원 이상)2구간(50만원 이상)제과·아이스크림 전문점5%-1만원이용금액 건당 1만원~5만원까지 할인 적용1일 1회, 최대 2,500원까지 월 할인한도 내에서 할인 적용 (주말, 공휴일 제외)제과·아이스크림전문 업종 (파리바게뜨, 뚜레쥬르, 배스킨라빈스 등) 
대중교통버스·지하철 5% 청구할인버스·지하철 청구할인구분할인율전월 이용실적에 따른 월 할인한도1구간(30만원 이상)2구간(50만원 이상)전국 버스·지하철5%1천5백원


09636
해피리워드 체크카드
https://img1.kbcard.

국민행복체크카드
https://img1.kbcard.com/ST/img/cxc/kbcard/upload/img/product/02066_img.png
[할인 서비스]병원 5% 할인
[할인 서비스]키즈카페/문화센터/학습지 5% 할인
[단체보험 무료가입]대중교통상해 사망/후유장애 보장 1억원
[단체보험 무료가입]태반조기박리진단비 10만원
[단체보험 무료가입]임신중독증진단비 10만원


   병원5% 환급 할인     KB국민행복카드 병원 할인 요약          구분 할인율 전월실적별 통합할인한도   30만원 이상 40만원 이상 50만원 이상     병원 5% 1천원 3천원 5천원     병원 : 종합병원, 일반/치과/한방병원, 일반/치과/한의원 업종								※ KB국민행복체크카드 할인서비스는 KB국민카드 업종분류 기준, 지정 업종에 한하여 제공됩니다.   최초 카드 사용등록일로부터 다음달 말일까지 ‘KB국민행복체크카드’ 이용실적이 30만원 미만 시에도 할인서비스가 제공됩니다.(단, 30만원 이상 구간 적용)
   교육5% 환급할인     KB국민행복카드 교육 할인 요약           구분 할인율 전월실적별 통합할인한도   30만원 이상 40만원 이상 50만원 이상     교육 키즈카페 5% 1천원 3천원 5천원   문화센터   학습지    교육  키즈카페 : 유아전문 교육기관/놀이기관 업종 문화센터 : 문화센터 업종 학습지 : 학습지 업종  ※ KB국민행복카드 할인서비스는 KB국민카드 업종분류 기준, 지정 업종에 한하여 제공됩니다. 최초 카드 사용등록일로부터 다음달 말일까지 KB국민행복카드 이용실적이 30만원 미만 시에도 할인서비스가 제공됩니다.(단, 30만원 이상 구간 적용)
   단체보험 무료가입     KB국민행복카드 단체보험 무료가입 요약       보장내용 보장금액     대중교통상해 사망 1억   대중교통상해 후유장해   태반조기박리진단 10만원   임신중독증진단    ※ 대중교통상해 후유장해는 장해지급률에 따라 지급 ※ 태반조기박리 및 임

In [197]:
cursor.close()
db.close()